# SnowEx Database Preview
![fireworks] (https://i.gifer.com/FH7W.gif "fireworks")

Learning Objectives:

* First taste of the database!
* Code snippets to extract and prep data.
* Generate ideas for project pitches.

## The Basics
### How are the data contained?
<img src='content/03_database-tables.png'>

## Set Up Computing Environment

In [ ]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt 
import datetime
import re
import geopandas as gpd

# some mapping widgets
import ipyleaflet
from ipyleaflet import Map, GeoData, Rectangle, basemaps, LayersControl, basemap_to_tiles, TileLayer, SplitMapControl, Polygon
import ipywidgets

#database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas

**POINTER -->** Notice where I import the four primary database tables. Can anyone call out what code line does this from the code block above?

In [ ]:
# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

print('snowexsql database successfully loaded!')

### What's the first thing you might like to do?
Find overlapping data for data analysis comparison

## Grand Mesa IOP (2020)
### Example 1: Let's find all the pits that overlap with an airborne sensor of interest!
First, it would be helpful to know, which of the airborne sensors are part of the database, right?

In [ ]:
# Query the session using .surveyors() to generate a list
qry = session.query(ImageData.surveyors)

# Locate all that are distinct
airborne_sensors_list = session.query(ImageData.surveyors).distinct().all()

print('list of airborne sensors by "surveyor" name: \n', airborne_sensors_list)

### 1a). Unsure of the flight date, but know which sensor you'd like to overlap with, here's how: 

In [ ]:
# Airborne sensor from list above
sensor = 'UAVSAR team, JPL'

# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == sensor)

# Grab the unique dates
qry = qry.distinct()

# Execute the query 
dates = qry.all() 

# Clean up the dates 
dates = [d[0] for d in dates] 
dlist = [str(d) for d in dates]
dlist = ", ".join(dlist)
print('%s flight dates are: %s' %(sensor, dlist))

# Find all the snow pits done on these days
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))

# Return a geopandas df
df = query_to_geopandas(qry, engine)

# View the returned pandas dataframe!
print(df.head())

# Close your session to avoid hanging transactions
session.close()

### 1b).Want to select an exact flight date match? Here's how:

In [ ]:
# Pick a day from the list of dates
dt = dates[0] 

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date == dt)

# Return a geopandas df
df_exact = query_to_geopandas(qry, engine)

print('%s pits overlap with %s on %s' %(len(df_exact), sensor, dt))

# View snows pits that align with first UAVSAR date
df_exact.head()

### 1c). Want to select a range of dates near the flight date? Here's how:

In [ ]:
# Form a date range to query on either side of our chosen day 
date_range = [dt + i * datetime.timedelta(days=1) for i in [-1, 0, 1]]

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))

# Return a geopandas df
df_range = query_to_geopandas(qry, engine)

# Clean up dates (for print statement only)
dlist = [str(d) for d in date_range]
dlist = ", ".join(dlist)

print('%s pits overlap with %s on %s' %(len(df_range), sensor, dlist))

# View snow pits that are +/- 1 day of the first UAVSAR flight date
df_range.sample(10)

### 1d). Have a known date that you wish to select data for, here's how:

In [ ]:
# Find all the data that was collected on 2-12-2020
dt = datetime.date(2020, 2, 12)

#--------------- Point Data -----------------------------------
# Grab all Point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
point_type = session.query(PointData.type).filter(PointData.date == dt).distinct().all()

# Clean up point data (i.e. remove tuple)
point_instruments = [p[0] for p in point_instruments]
point_instruments = ", ".join(point_instruments)
point_type = [p[0] for p in point_type]
point_type = ", ".join(point_type)
print('Point data on %s are: %s, with the following list of parameters: %s' %(str(dt), point_instruments, point_type))

#--------------- Layer Data -----------------------------------
# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument).filter(LayerData.date == dt).distinct().all()
layer_type = session.query(LayerData.type).filter(LayerData.date == dt).distinct().all()

# Clean up layer data 
layer_instruments = [l[0] for l in layer_instruments if l[0] is not None]
layer_instruments = ", ".join(layer_instruments)
layer_type = [l[0] for l in layer_type]
layer_type = ", ".join(layer_type)
print('\nLayer Data on %s are: %s, with the following list of parameters: %s' %(str(dt), layer_instruments, layer_type))

#--------------- Image Data -----------------------------------
# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
image_type = session.query(ImageData.type).filter(ImageData.date == dt).distinct().all()

# Clean up image data (i.e. remove tuple)
image_instruments = [i[0] for i in image_instruments]
image_instruments = ", ".join(image_instruments)
image_type = [i[0] for i in image_type]
image_type = ", ".join(image_type)
print('\nImage Data on %s are: %s, with the following list of parameters: %s' %(str(dt), image_instruments, image_type))

## Time Series (2020)

In [ ]:
# Set up database
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas

In [ ]:
# Query temp profiles from 2 sites: Banner Forest, Banner Snotel

In [ ]:
# Plot full timeseries of temp profile vs depth in side by side plot 

## Explore the Spatial Extent of Field Campaign Data

In [ ]:
bbox = [-125, 49, -102, 31]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

In [ ]:
m = Map(basemap=basemaps.CartoDB.Positron, center=bbox_ctr, zoom=4)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

# more info on available basemaps here: https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html

### Query the Database to add Spatial Data to Our Map

In [ ]:
# # database imports
# from snowexsql.db import get_db
# from snowexsql.data import PointData, LayerData, ImageData, SiteData
# from snowexsql.conversions import query_to_geopandas, query_to_pandas

In [ ]:
# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

### Let's find out where we have liquid water content (LWC) data 

**Pointer -->**  LWC data is in the LayerData table, because data at a single location were measured as a profile on the pit wall face (i.e has a vertical dimension) 

In [ ]:
# query the LayerData for all LWC values
# qry = session.query(LayerData).filter(LayerData.type == 'lwc_vol')

# query the geometry property of PointData
# qry = session.query(LayerData.geom).distinct()

# query the LayerData for all LWC values
qry = session.query(LayerData.longitude, LayerData.latitude).filter(LayerData.type == 'lwc_vol').distinct() # 

# # query the geometry property of PointData
# qry = query(LayerData.geom).distinct()

# # limit the number of entries
# qry = qry.limit(2000)

# convert query to geopandas df
df = query_to_pandas(qry, engine)

df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
print(type(df))

# # convert query to geopandas df
# df = query_to_geopandas(qry, engine)

print(df.head())

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

session.close()

### Let's add these points to our map!

In [ ]:
m = Map(basemap=basemaps.CartoDB.Positron, center=bbox_ctr, zoom=4)

# geo_data = GeoData(geo_dataframe = df)

geo_data = GeoData(geo_dataframe = df,
    style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'lwc obs.')

m.add_layer(geo_data) 
m.add_control(LayersControl())
m

## grab geojson from gist

In [ ]:
import geopandas as gpd
import requests
url = "https://gist.githubusercontent.com/meganmason/21e6de1f7487b3e7defbe60ddde07e0e/raw/06939d8fffc0539a6014ee0d7c0c1d58938d9e8d/pits_raw.geojson"
gdf = gpd.read_file(url)
gdf.head()

## Recap

## References 